In [145]:
import numpy as np
import pandas as pd

In [146]:
def printTable(table):
    '''A simple way to display matrices in a more readable format'''
    for i in table:
        print(i)

In [147]:
def test_build(players, budget, inc, limits):
    nPlayers = len(players) 
    
    combos = {}
    combos_traceback = {}
    
    for forwards in range(limits["C"] + 1):
        for defense in range(limits["D"] + 1):
            for goalies in range(limits["G"] + 1):
                combos[(forwards, defense, goalies)] = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
                combos_traceback[(forwards, defense, goalies)] = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]
                
                
    ## First, fill in the base case for 0 forwards, defenders, and goalies
    combos[(0,0,0)] = [[-0 for i in range(budget + 1)] for i in range(nPlayers + 1)]
    
    ## Now we iterate through the combinations of forwards, defenders, and goalies
    for forwards in range(limits["C"] + 1):
        for defense in range(limits["D"] + 1):
            for goalies in range(limits["G"] + 1):
                
                
                roster_limits = {"C":forwards, "D": defense, "G":goalies}
                table = combos[(forwards, defense, goalies)]
                traceback = combos_traceback[(forwards, defense, goalies)]
                
                ## Filling in the base case for 0 players to choose from
                for i in range(budget + 1):
                    table[0][i] = 0
                    
                ## Now we incrememnt through each player
                for player_row in range(1, nPlayers + 1):
                    
                    ## For each player, we increment through our budget
                    for budget_interval in range(budget + 1):
                        
                        ##For each acceptable player, we see if we can afford the player and if we have roster space
                        player_position = players[player_row -1][3]
                        if roster_limits[player_position] > 0 and budget_interval >= players[player_row - 1][1]:
                    
                    
                            ## We calculate the points
                            points = players[player_row - 1][2]
                        
                            ## We look through previous dictionarys to find the rest of the ROI
                            if player_position == "C":
                                restOfROI = combos[(forwards - 1, defense, goalies)][player_row - 1][budget_interval - players[player_row - 1][1]]
                            elif player_position == "D":
                                restOfROI = combos[(forwards, defense - 1, goalies)][player_row - 1][budget_interval - players[player_row - 1][1]]
                            elif player_position == "G":
                                restOfROI = combos[(forwards, defense, goalies - 1)][player_row - 1][budget_interval - players[player_row - 1][1]]
                                
                            ## We compare it for the optimal value if we dont sign the player
                            noSign = table[player_row - 1][budget_interval]
                
                            ## if signing the player generates more profit than not signing the player, then we add him to the table
                            if points + restOfROI > noSign:
                                table[player_row][budget_interval] = points + restOfROI
                                traceback[player_row][budget_interval] = True
                                
                            else:
                                table[player_row][budget_interval] = noSign
                                traceback[player_row][budget_interval] = False
                                
                        ## if we cant afford the player or have roster room, we treat it as a noSign
                        else:
                            table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                            traceback[player_row][budget_interval] = False
                            
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    val = nPlayers
    i = -1
    forwards = limits["C"]
    defense = limits["D"]
    goalies = limits["G"]
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while val > 0:
        
        ## If our traceback table tells us to buy the investment
        if combos_traceback[(forwards, defense, goalies)][val][i]:
            
            ## We record the player position
            player_position = players[val - 1][3]
            
            ## We add our investment number, cost, and ROI to our output list
            ret_list.append((players[val - 1][0], players[val - 1][1] * inc, players[val - 1][2], players[val - 1][3]))
            
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            ## and incrememnt back the roster space
            i -= players[val - 1][1]
            val -= 1
            
            if player_position == "C":
                forwards -= 1
            elif player_position == "D":
                defense -= 1
            elif player_position == "G":
                goalies -= 1
        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            val -= 1
    
    
    return ret_list, combos, combos_traceback

In [148]:
test_items = [("Mackinon", 12, 60, "C"),("Rantanen", 10, 50, "C"),("Makar", 1, 30,"C"),("Girard", 8, 25, "D"), ("Kadri", 6, 35, "D"), ("Landeskog",9, 45, "C")]
limits = {"C": 2, "D": 2, "G": 0}


#test_items = [1,2,3,4,5]
results = test_build(test_items, 24, 1, limits)
#printTable(test_tables[2])

results[0]

# for key, value in results[1].items():
#     print(key)
#     printTable(value)
#     print()
    

[('Landeskog', 9, 45, 'C'),
 ('Kadri', 6, 35, 'D'),
 ('Girard', 8, 25, 'D'),
 ('Makar', 1, 30, 'C')]

In [112]:
def buildTable(players, budget, inc):
    nPlayers = len(players)
    table = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
    traceback = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]
    
    ## Now we fill in our base cases for we have no players to chose from
    for i in range(budget + 1):
        table[0][i] = 0
        
    ## Now we increment through every player
    for player_row in range(1,nPlayers + 1):
        
        ## for each player, we increment through our budget
        for budget_interval in range(budget + 1):
            
            ## first, we see if we can afford the player in this budget range and if we have room for the player
                #player_position = players[player_row - 1][3]
                #if budget_interval >= players[player_row - 1][1] and limits[player_position] > 0:
                if budget_interval >= players[player_row - 1][1]:
                    
                    ## We calculate the points and rest of ROI
                    points = players[player_row - 1][2]
                    
                    restOfROI = table[player_row - 1][budget_interval - players[player_row - 1][1]]
                    
                    ## We compare it for the optimal value if we dont sign the player
                    noSign = table[player_row - 1][budget_interval]
                    
                    ## if signing the player generates more profit than not signing the player, then we add him to the table
                    if points + restOfROI > noSign:
                        table[player_row][budget_interval] = points + restOfROI
                        traceback[player_row][budget_interval] = True
                        
                        
                        
                    else:
                        table[player_row][budget_interval] = noSign
                        traceback[player_row][budget_interval] = False
                        
            ## if we cant afford the player, we treat it as a noSign
                else:
                    table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                    traceback[player_row][budget_interval] = False
        
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    val = nPlayers
    i = -1
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while val > 0:
        
        ## If our traceback table tells us to buy the investment
        if traceback[val][i]:
            
            ## We add our investment number, cost, and ROI to our output list
            ret_list.append((players[val - 1][0], players[val - 1][1] * inc, players[val - 1][2], players[val - 1][3]))
            
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            i -= players[val - 1][1]
            val -= 1
        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            val -= 1
    
    
    return table, traceback, ret_list


In [113]:
test_items = [("Mackinon", 12, 60, "C"),("Rantanen", 10, 50, "C"),("Makar", 1, 30,"C"),("Girard", 8, 25, "D"), ("Kadri", 6, 35, "D"), ("Landeskog",9, 45, "C")]
limits = {"C": 2, "D": 1}


#test_items = [1,2,3,4,5]
test_tables = buildTable(test_items, 20, 1)
printTable(test_tables[2])

TypeError: buildTable() missing 1 required positional argument: 'limits'

In [ ]:
def buildTable(players, budget, inc):
    nPlayers = len(players)
    table = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
    traceback = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]
    
    ## Now we fill in our base cases for we have no players to chose from
    for i in range(budget + 1):
        table[0][i] = 0
        
    ## Now we increment through every player
    for player_row in range(1,nPlayers + 1):
        
        ## for each player, we increment through our budget
        for budget_interval in range(budget + 1):
            
            ## first, we see if we can afford the player in this budget range and if we have room for the player
                #player_position = players[player_row - 1][3]
                #if budget_interval >= players[player_row - 1][1] and limits[player_position] > 0:
                if budget_interval >= players[player_row - 1][1]:
                    
                    ## We calculate the points and rest of ROI
                    points = players[player_row - 1][2]
                    
                    restOfROI = table[player_row - 1][budget_interval - players[player_row - 1][1]]
                    
                    ## We compare it for the optimal value if we dont sign the player
                    noSign = table[player_row - 1][budget_interval]
                    
                    ## if signing the player generates more profit than not signing the player, then we add him to the table
                    if points + restOfROI > noSign:
                        table[player_row][budget_interval] = points + restOfROI
                        traceback[player_row][budget_interval] = True
                        
                        
                        
                    else:
                        table[player_row][budget_interval] = noSign
                        traceback[player_row][budget_interval] = False
                        
            ## if we cant afford the player, we treat it as a noSign
                else:
                    table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                    traceback[player_row][budget_interval] = False
        
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    val = nPlayers
    i = -1
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while val > 0:
        
        ## If our traceback table tells us to buy the investment
        if traceback[val][i]:
            
            ## We add our investment number, cost, and ROI to our output list
            ret_list.append((players[val - 1][0], players[val - 1][1] * inc, players[val - 1][2], players[val - 1][3]))
            
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            i -= players[val - 1][1]
            val -= 1
        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            val -= 1
    
    
    return table, traceback, ret_list


In [4]:
def optimize(items, money, increment):
    ''' This function carries out the dynamic programming solution to a 
    list of possible investments, budget, and money increment'''
    
    ## Since we might be working with large amounts of money, we 
    ## Use a given increment to ensure that the price changes
    ## in meaningful steps only
    money = int(money / increment)
    for item in items:
        item[1] = int(item[1] / increment)
    
    ## First we use the list of items and money allowed to build
    ## the value table and traceback table
    nItems = len(items)
    table = [[-1 for i in range(money + 1)] for i in range(nItems + 1)]
    traceback = [[None for i in range(money + 1)] for i in range(nItems + 1)]
    
    ## Now we fill in our base cases for when we have no players to choose from
    for i in range(money + 1):
        table[0][i] = 0
    
    ## Now we fill in our table using Dynamic Programming
    ## we start by incrementing through every player
    for itemNum in range(1,nItems + 1):

        ## with every player, we increment through our budget
        for j in range(money + 1):
            
            ## first we check to see if the cost of the player is
            ## less than our budget increment
            if int(items[itemNum - 1][1]) <= j:

                ## For ease of use, we explicitly record the ROI of the investment
                valItem = int(items[itemNum - 1][2])
                
                ## Then we calculate and record the best ROI of our remaining money
                restOfROI = table[itemNum - 1][j - int(items[itemNum - 1][1])] 
                
                ## We compare it to the optimal value of not buying the investment,
                ## which is basically the optimal value between all previous investments
                noBuy = table[itemNum - 1][j]

                ## If the value of buying the investment and optimizing the remaining money
                ## is greater than the value of not buying and investing in previous options,
                ## We record this newly calculated optimal value in our table record our traceback to buy
                if valItem + restOfROI >= noBuy:
                    table[itemNum][j] = valItem + restOfROI
                    traceback[itemNum][j] = True
                    
                ## Otherwise, we record the optimized value of previous investments in our table
                ## and mark a false in our traceback table
                else:
                    table[itemNum][j] = noBuy
                    traceback[itemNum][j] = False
                    
            ## If we can not afford the investment, we have to treat it as if we
            ## chose not to make the investment in terms of our table and traceback table
            else:
                table[itemNum][j] = table[itemNum - 1][j]
                traceback[itemNum][j] = False



    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    val = nItems
    i = -1
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while val > 0:
    
        ## If our traceback table tells us to buy the investment
        if traceback[val][i]:

            ## We add our investment number, cost, and ROI to our output list
            ret_list.append((items[val - 1][0], items[val - 1][1] * increment, items[val - 1][2]))
            
            ## We increment our money iteration value backwards
            ## by the amount that we "spent" on the investment
            i -= int(items[val - 1][1])

            ## And increment our item marker value back by 1
            val -= 1

        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            val -= 1
            
            
    print("Total Optimized ROI: " + str(table[nItems][money]))
    for inv in ret_list:
        print("Investment Name: " + inv[0] + ",\t Investment Cost: " + str(inv[1]) + ",\t Investment ROI: " + str(inv[2]))
    ## We return our total ROI and our return list
    
    return table[nItems][money], ret_list